# Import Libraries

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv

# Import Data

In [2]:
load_dotenv()

db_user = os.environ.get("DB_USER")
db_password = os.environ.get("DB_PASSWORD")
db_host = os.environ.get("DB_HOST")

js_db_name = os.environ.get("JS_DB_NAME")
lc_db_name = os.environ.get("LC_DB_NAME")

# Use the credentials to connect to the database
# Database connection string
js_conn_string = f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{js_db_name}"
lc_conn_string = f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{lc_db_name}"

# Create engine
js_engine = create_engine(js_conn_string)
lc_engine = create_engine(lc_conn_string)

In [3]:
# Jotstar Tables
js_subscribers = pd.read_sql_query("SELECT * FROM subscribers", js_engine, 
                                   parse_dates={"subscription_date":'%Y-%m-%d', "last_active_date":'%Y-%m-%d', "plan_change_date":'%Y-%m-%d'})

In [4]:
# LioCinema Tables
lc_subscribers = pd.read_sql_query("SELECT * FROM subscribers", lc_engine, 
                                   parse_dates={"subscription_date":'%Y-%m-%d', "last_active_date":'%Y-%m-%d', "plan_change_date":'%Y-%m-%d'})

# Define Change Type

In [5]:
js_plan_hierarchy = {'Free':1, 'VIP':2, 'Premium':3}
lc_plan_hierarchy = {'Free':1, 'Basic':2, 'Premium':3}

In [6]:
def change_type(row, plan_hierarchy):
    if pd.notna(row['plan_change_date']) & (plan_hierarchy.get(row['new_subscription_plan'], 0) > plan_hierarchy.get(row['subscription_plan'], 0)):
        return 'Upgrade'
    elif pd.notna(row['plan_change_date']) & (plan_hierarchy.get(row['new_subscription_plan'], 0) < plan_hierarchy.get(row['subscription_plan'], 0)):
        return 'Downgrade'
    else:
        return 'No Change'

In [7]:
js_subscribers['change_type'] = js_subscribers.apply(change_type, axis=1, args=(js_plan_hierarchy,))
lc_subscribers['change_type'] = lc_subscribers.apply(change_type, axis=1, args=(lc_plan_hierarchy,))

# Revenue Analysis

## Fill na values in last_active_date with '2024-12-31'

- If the user is active their last_active_date is null as per metadata 
- It's difficult to find the revenue generated from the user, so assuming '2024-12-31' as their last_active_date. Also maximum plan_change_date is '2024-12-31'
- Fill missing last_active_date with 31 Dec 2024 (assuming still active)

In [8]:
js_subscribers['last_active_date'].fillna(pd.Timestamp('2024-12-31'), inplace=True)
lc_subscribers['last_active_date'].fillna(pd.Timestamp('2024-12-31'), inplace=True)

## Platform-Wise Monthly Prices 

In [9]:
js_prices = {'VIP': 159, 'Premium': 359}
lc_prices = {'Basic': 69, 'Premium': 129}

## Consider Subscribers who started with Free as their Initial Plan during Revenue Calculation

Because they might get upgraded to basic/premium and its revenue should also be considered for analysis

In [10]:
js_subscribers[js_subscribers['subscription_plan']=='Free'].shape, lc_subscribers[lc_subscribers['subscription_plan']=='Free'].shape

((12096, 9), (104992, 9))

## Calculate Revenue

In [11]:
def calculate_revenue(row, price):
    if pd.notna(row['plan_change_date']): # If user changed plan
        # Initial Plan Duration
        initial_plan_days = (row['plan_change_date'] - row['subscription_date']).days
        initial_plan_months = initial_plan_days / 30
        # Total Initial Plan Revenue
        initial_plan_rev = initial_plan_months * price.get(row['subscription_plan'], 0)
        
        # New Plan Duration
        new_plan_days = ((row['last_active_date'] - row['plan_change_date']).days) + 1 # Add 1 to capture last active day 
        new_plan_months = new_plan_days / 30
        # Total New Plan Revenue
        new_plan_rev = new_plan_months * price.get(row['new_subscription_plan'], 0)
        
        return initial_plan_rev + new_plan_rev
    else:
        active_days = ((row['last_active_date'] - row['subscription_date']).days) + 1 
        active_months = active_days / 30
        return active_months * price.get(row['subscription_plan'], 0)

In [12]:
js_subscribers['revenue'] = js_subscribers.apply(calculate_revenue, axis=1, args=(js_prices,))
lc_subscribers['revenue'] = lc_subscribers.apply(calculate_revenue, axis=1, args=(lc_prices,))

In [13]:
js_total_revenue = js_subscribers['revenue'].sum()
lc_total_revenue = lc_subscribers['revenue'].sum()

print('Total Revenue')
print(f"JotStar's: {js_total_revenue/1000000:.2f}M")
print(f"LioCinema's: {lc_total_revenue/1000000:.2f}M")

Total Revenue
JotStar's: 51.80M
LioCinema's: 20.46M


## Revenue generated from Subsricber who upgraded from Free plan

In [14]:
js_subscribers[(js_subscribers['subscription_plan']=='Free') & (js_subscribers['change_type']=='Upgrade')].head()

,user_id,age_group,city_tier,subscription_date,subscription_plan,last_active_date,plan_change_date,new_subscription_plan,change_type,revenue
76,UIDJS006315e24bf,45+,Tier 1,2024-01-04,Free,2024-12-31,2024-03-04,Premium,Upgrade,3625.900000
81,UIDJS006711c9331,25-34,Tier 1,2024-05-22,Free,2024-12-31,2024-06-22,VIP,Upgrade,1022.900000
89,UIDJS007b5054731,25-34,Tier 1,2024-03-19,Free,2024-12-31,2024-09-19,VIP,Upgrade,551.200000
106,UIDJS008fe1ea9ab,25-34,Tier 1,2024-02-23,Free,2024-12-31,2024-09-23,Premium,Upgrade,1196.666667
151,UIDJS00d8867463a,25-34,Tier 3,2024-08-25,Free,2024-12-31,2024-09-25,Premium,Upgrade,1172.733333


Demo:

js_monthly_prices = {'VIP': 159, 'Premium': 359}

Let's consider user with index 76
- subscription_plan: Free
- new_subscription_plan: Premium
- subscription_date: 2024-01-04 (04 Jan 2024)
- plan_change_date: 2024-03-04 (04 Mar 2024)
- last_active_date: 2024-12-31 (31 Dec 2024)

old_plan_duration = (04 Mar 2024) - (04 Jan 2024) = 60 days (doesn't including plan_change_date) => 2 Months

old_plan_rev = 2 * 0 => 0

new_plan_duration = (31 Dec 2024) - (04 Mar 2024) + 1 = 303 days (doesn't including plan_change_date) => 10.1 Months

new_plan_rev = 10.1 * 359 => 3625.9